In [7]:
import re
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

year = 2019
df = pd.read_csv('./gaon_final_data.csv', index_col=0)
df = df.loc[df['year']==year]

# 노래제목 리스트
music_list = df['music'].unique().tolist()
# music_list = music_list[100:]

# singer
for music in music_list:
    try:
        singer = df['singer'].loc[df['music']==music].tolist()[0]
    except:
        print('오류', music)


# 노래 id 딕셔너리 만들기 (key:노래id값, value:노래 제목)
id_dict={}
for music in music_list:
    singer = df['singer'].loc[df['music']==music].tolist()[0]

    # 가수이름 - 정규표현식으로 괄호 안의 내용 모두 삭제
    if singer.find('(') != -1:
        singer = re.sub('\([^)]*\)','', singer) 
    else:
        pass

    # 노래제목 - 정규표현식으로 괄호 안의 내용 모두 삭제
    if music.find('(') != -1:
        music = re.sub('\([^)]*\)','', music) 
    else:
        pass
    
    singer = singer.rstrip()
    music = music.rstrip()

    # 공백을 '+'로 바꿔줌
    music = music.replace(' ', '+')
    singer = singer.replace(' ', '+')

    # 특수문자 제거
    singer = re.sub('[-=+#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', "",singer)

    print(f'music:{music}, singer:{singer}')

    try:
        URL = f'https://www.melon.com/search/total/index.htm?q={music}+{singer}'
        header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
        response = requests.get(URL,headers=header)
        time.sleep(1)
        soup = BeautifulSoup(response.text, 'html.parser')

        # try:
        links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
        #frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input

        music_id = links.attrs['value']
        print(music_id)
    except:
        print(f'오류, music:{music}')
        pass

    # +을 공백로 바꿔줌
    music = music.replace('+', ' ')
    singer = singer.replace('+', ' ')

    id_dict[music_id] = [music, singer]
    time.sleep(1)

# 노래 id만 있는 리스트 
id_list = list(id_dict.keys())



# 데이터 뽑아오기
result = []

for id in id_list:
    music = id_dict[id][0]
    singer = id_dict[id][1]
    print(f'현재 {music} 크롤링 중')

    try:
        URL_1 = f'https://www.melon.com/song/detail.htm?songId={id}'
        response_1 = requests.get(URL_1,headers=header)
        time.sleep(1)
        soup = BeautifulSoup(response_1.text, 'html.parser')

        # 발매일, 장르
        release_date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
        category = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
        

        # 좋아요 개수 json 접근
        URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={id}'
        response_2 = requests.get(URL_2,headers=header).text
        time.sleep(1)
        like_json = json.loads(response_2)
        music_like = like_json['contsLike'][0]['SUMMCNT']

        # 코멘트 개수 json 접근
        URL_3 = f'https://www.melon.com/song/songReviewCnt.json?songId={id}'
        response_3 = requests.get(URL_3,headers=header).text
        time.sleep(1)

        comment_json = json.loads(response_3)
        comment_count = comment_json['reviewValidCmtCnt']

        temp = {
        'music':music,
        'singer':singer,
        'music_id':id,
        'release_date':release_date,
        'category':category,
        'music_like':music_like,
        'comment_count':comment_count,
    }

        result.append(temp)
    except:
        print(f'오류, music:{music}')
        pass


df = pd.DataFrame(result)
df.to_csv(f'./melon_data_{year}.csv')



music:180도, singer:벤
31477685
music:넘쳐흘러, singer:엠씨더맥스
31532643
music:SOLO, singer:제니
31417871
music:신용재, singer:하은
31406357
music:벌써+12시, singer:청하
31532438
music:MILLIONS, singer:위너
31506637
music:아낙네, singer:MINO
31448480
music:너를+만나, singer:폴킴
31388145
music:YES+or+YES, singer:TWICE
31403163
music:첫눈에, singer:헤이즈
31495659
music:모든+날,+모든+순간, singer:폴킴
30962526
music:내+생에+아름다운, singer:케이윌
31373277
music:Love+Shot, singer:EXO
31492319
music:가을+타나+봐, singer:바이브
31314144
music:삐삐, singer:아이유
31346009
music:습관, singer:숀
31526249
music:thank+u,+next, singer:ArianaGrande
31402611
music:고백, singer:양다일
31356458
music:Way+Back+Home, singer:숀
31151836
music:흔한+이별, singer:허각
31453551
music:아름답고도+아프구나, singer:비투비
31417922
music:열애중, singer:벤
31062863
music:하루도+그대를+사랑하지+않은+적이+없었다, singer:임창정
31316695
music:Tempo, singer:EXO
31399721
music:이별하러+가는+길, singer:임한별
31304766
music:IDOL, singer:방탄소년단
31266290
music:Good+Day, singer:pH1,KidMilli,루피
31340985
music:Snowman, singer:Sia
30717645
music:사계, si

KeyboardInterrupt: 